# 1.5 練習問題

In [1]:
ENV["LINES"] = 10
ENV["COLUMNS"] = 1000

1000

In [38]:
using DataFrames, DataFramesMeta, CSV, Statistics

## 1.5.1

In [31]:
turnout_df = CSV.read("../../data/INTRO/bias-in-turnout/data/turnout.csv", DataFrame, missingstring=["NA"])

,year,VEP,VAP,total,ANES,felons,noncit,overseas,osvoters
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64?
1,1980,159635,164445,86515,71,802,5756,1803,missing
2,1982,160467,166028,67616,60,960,6641,1982,missing
3,1984,167702,173995,92653,74,1165,7482,2361,missing
4,1986,170396,177922,64991,53,1367,8362,2216,missing
5,1988,173579,181955,91595,70,1594,9280,2257,missing
6,1990,176629,186159,67859,47,1901,10239,2659,missing
7,1992,179656,190778,104405,75,2183,11447,2418,missing
8,1994,182623,195258,75106,56,2441,12497,2229,missing
9,1996,186347,200016,96263,73,2586,13601,2499,missing


In [32]:
describe(turnout_df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Int64,Float64,Int64,Int64,Type
1,year,1993.14,1980,1993.0,2008,0,Int64
2,VEP,1.8264e5,159635,1.8114e5,213314,0,Int64
3,VAP,1.94226e5,164445,193018.0,230872,0,Int64
4,total,89778.3,64991,89055.0,131304,0,Int64
5,ANES,65.7857,47,70.5,78,0,Int64
6,felons,2176.64,802,2312.0,3168,0,Int64
7,noncit,12229.1,5756,11972.0,19392,0,Int64
8,overseas,2745.71,1803,2458.5,4972,0,Int64
9,osvoters,263.0,263,263.0,263,13,"Union{Missing, Int64}"


In [33]:
turnout_df = @chain turnout_df begin
    @rtransform begin
        @astable begin
            :est_vote_VAP = :total / (:VAP + :overseas - :felons - :noncit) * 100
            :est_vote_VEP = :total / :VEP * 100
        end
    end
end

,year,VEP,VAP,total,ANES,felons,noncit,overseas,osvoters,est_vote_VAP,est_vote_VEP
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64?,Float64,Float64
1,1980,159635,164445,86515,71,802,5756,1803,missing,54.1768,54.1955
2,1982,160467,166028,67616,60,960,6641,1982,missing,42.1522,42.137
3,1984,167702,173995,92653,74,1165,7482,2361,missing,55.2463,55.2486
4,1986,170396,177922,64991,53,1367,8362,2216,missing,38.1382,38.1412
5,1988,173579,181955,91595,70,1594,9280,2257,missing,52.8418,52.7685
6,1990,176629,186159,67859,47,1901,10239,2659,missing,38.4083,38.4189
7,1992,179656,190778,104405,75,2183,11447,2418,missing,58.143,58.1138
8,1994,182623,195258,75106,56,2441,12497,2229,missing,41.1429,41.1263
9,1996,186347,200016,96263,73,2586,13601,2499,missing,51.6632,51.6579


In [34]:
@chain turnout_df begin
    @rtransform begin
        :diff_vote_VAP = :est_vote_VAP - :ANES
        :diff_vote_VEP = :est_vote_VEP - :ANES
    end
    @select(:diff_vote_VAP, :diff_vote_VEP)
    describe
end

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,diff_vote_VAP,-16.8151,-22.486,-16.8401,-8.59171,0,Float64
2,diff_vote_VEP,-16.8363,-22.4894,-16.8927,-8.58105,0,Float64


In [68]:
@chain turnout_df begin
    @rtransform begin
        :diff_vote_VAP = :est_vote_VAP - :ANES
        :diff_vote_VEP = :est_vote_VEP - :ANES
        :is_president_voting = rem(:year, 4) == 0 ? 1 : 0
    end
    @select(:is_president_voting, :diff_vote_VAP, :diff_vote_VEP)
    groupby(:is_president_voting)
    @combine begin
        :mean_diff_vote_VAP = mean(:diff_vote_VAP)
        :mean_diff_vote_VEP = mean(:diff_vote_VEP)
    end
end

,is_president_voting,mean_diff_vote_VAP,mean_diff_vote_VEP
,Int64,Float64,Float64
1,0,-15.4226,-15.4288
2,1,-17.8595,-17.892


In [65]:
@chain turnout_df begin
    @rtransform begin
        :diff_vote_VAP = :est_vote_VAP - :ANES
        :diff_vote_VEP = :est_vote_VEP - :ANES
    end
    @transform begin
        :is_first = ifelse.(:year .<= mean(:year), 1, 0)
    end
    @select(:is_first, :diff_vote_VAP, :diff_vote_VEP)
    groupby(:is_first)
    @combine begin
        :mean_diff_vote_VAP = mean(:diff_vote_VAP)
        :mean_diff_vote_VEP = mean(:diff_vote_VEP)
    end
end

,is_first,mean_diff_vote_VAP,mean_diff_vote_VEP
,Int64,Float64,Float64
1,0,-17.7883,-17.8189
2,1,-15.8419,-15.8538


In [67]:
@chain turnout_df begin
    @rsubset(:year == 2008)
    @rtransform begin
        :vote_VAP = :total / (:VAP + :overseas) * 100
        :vote_VEP = :total / (:VEP) * 100
        :vote_adj_VAP = (:total - :osvoters) / (:VAP - :felons - :noncit) * 100
    end
    @select(:ANES, :vote_VAP, :vote_VEP, :vote_adj_VAP)
end

,ANES,vote_VAP,vote_VEP,vote_adj_VAP
,Int64,Float64,Float64,Float64
1,78,55.6741,61.5543,62.8992


## 1.5.2

In [69]:
kenya_df = CSV.read("../../data/INTRO/population-dynamics/data/Kenya.csv", DataFrame, missingstring=["NA"])
sweden_df = CSV.read("../../data/INTRO/population-dynamics/data/Sweden.csv", DataFrame, missingstring=["NA"])
world_df = CSV.read("../../data/INTRO/population-dynamics/data/World.csv", DataFrame, missingstring=["NA"])

,country,period,age,births,deaths,py.men,py.women
,String7,String15,String7,Float64,Float64,Float64,Float64
1,WORLD,1950-1955,0-4,0.0,1.0109e5,9.46911e5,9.04909e5
2,WORLD,1950-1955,5-9,0.0,7960.86,7.26903e5,6.94575e5
3,WORLD,1950-1955,10-14,0.0,5550.18,6.66795e5,6.35492e5
4,WORLD,1950-1955,15-19,54238.3,5831.19,6.26191e5,6.00678e5
5,WORLD,1950-1955,20-24,1.31939e5,6649.43,5.73518e5,555221.0
6,WORLD,1950-1955,25-29,1.28063e5,6425.22,5.085e5,507277.0
7,WORLD,1950-1955,30-34,89246.6,6208.63,4.33338e5,4.37132e5
8,WORLD,1950-1955,35-39,56005.5,6877.15,4.00279e5,4.05528e5
9,WORLD,1950-1955,40-44,24326.6,7995.13,3.73771e5,3.82444e5


In [76]:
function calculate_CBR(df)::DataFrame
    df = @chain df begin
        rename("py.men" => :py_men, "py.women" => :py_women)
        @rtransform :py = :py_men + :py_women
        groupby(:period)
        @combine begin
            :births_total = sum(:births)
            :py_total = sum(:py)
        end
        @rtransform :CBR = :births_total / :py_total
    end
    return df
end

println(calculate_CBR(kenya_df))
println(calculate_CBR(sweden_df))
println(calculate_CBR(world_df))

2×4 DataFrame
 Row │ period     births_total  py_total       CBR       
     │ String15   Float64       Float64        Float64   
─────┼───────────────────────────────────────────────────
   1 │ 1950-1955       1672.49  32104.7        0.0520949
   2 │ 2005-2010       7275.25      1.88894e5  0.0385151
2×4 DataFrame
 Row │ period     births_total  py_total  CBR       
     │ String15   Float64       Float64   Float64   
─────┼──────────────────────────────────────────────
   1 │ 1950-1955       548.069   35597.8  0.0153961
   2 │ 2005-2010       546.474   45823.9  0.0119255


2×4 DataFrame
 Row │ period     births_total  py_total   CBR       
     │ String15   Float64       Float64    Float64   
─────┼───────────────────────────────────────────────
   1 │ 1950-1955     4.88892e5  1.3097e7   0.0373286
   2 │ 2005-2010     6.74581e5  3.33688e7  0.0202159


In [90]:
function calc_total_ASFR(df)::DataFrame
    f(x) = 1 <= parse(Int64, split(replace(x, "+"=>"-"), "-")[1]) / 15 <= 3
    df = @chain df begin
        rename("py.men" => :py_men, "py.women" => :py_women)
        @rsubset(f(:age))
        groupby(:period)
        @combine begin
            :births_total = sum(:births)
            :py_women_total = sum(:py_women)
        end
        @rtransform :ASFR = :births_total / :py_women_total
    end
    return df
end
println(calc_total_ASFR(kenya_df))
println(calc_total_ASFR(sweden_df))
println(calc_total_ASFR(world_df))

2×4 DataFrame
 Row │ period     births_total  py_women_total  ASFR     
     │ String15   Float64       Float64         Float64  
─────┼───────────────────────────────────────────────────
   1 │ 1950-1955       1672.49         7131.05  0.234537
   2 │ 2005-2010       7275.25        45946.3   0.158342
2×4 DataFrame
 Row │ period     births_total  py_women_total  ASFR      
     │ String15   Float64       Float64         Float64   
─────┼────────────────────────────────────────────────────
   1 │ 1950-1955       548.069         8724.88  0.0628168
   2 │ 2005-2010       546.474        10308.4   0.0530123
2×4 DataFrame
 Row │ period     births_total  py_women_total  ASFR      
     │ String15   Float64       Float64         Float64   
─────┼────────────────────────────────────────────────────
   1 │ 1950-1955     4.88892e5       3.22217e6  0.151728
   2 │ 2005-2010     6.74581e5       8.68057e6  0.0777116


In [93]:
function calc_TFR(df)::DataFrame
    f(x) = 1 <= parse(Int64, split(replace(x, "+"=>"-"), "-")[1]) / 15 <= 3
    df = @chain df begin
        rename("py.men" => :py_men, "py.women" => :py_women)
        @rsubset(f(:age))
        groupby([:period, :age])
        @combine begin
            :births_total = sum(:births)
            :py_women_total = sum(:py_women)
        end
        @rtransform :ASFR = :births_total / :py_women_total
        groupby(:period)
        @combine :TFR = sum(:ASFR * 5)
    end
    return df
end
# (calc_TFR(kenya_df))
println(calc_TFR(kenya_df))
println(calc_TFR(sweden_df))
println(calc_TFR(world_df))

2×2 DataFrame
 Row │ period     TFR     
     │ String15   Float64 
─────┼────────────────────
   1 │ 1950-1955  7.59141
   2 │ 2005-2010  4.87957
2×2 DataFrame
 Row │ period     TFR     
     │ String15   Float64 
─────┼────────────────────
   1 │ 1950-1955  2.22692
   2 │ 2005-2010  1.90276
2×2 DataFrame
 Row │ period     TFR     
     │ String15   Float64 
─────┼────────────────────
   1 │ 1950-1955  5.00725
   2 │ 2005-2010  2.54362


In [94]:
function calculate_CDR(df)::DataFrame
    df = @chain df begin
        rename("py.men" => :py_men, "py.women" => :py_women)
        @rtransform :py = :py_men + :py_women
        groupby(:period)
        @combine begin
            :deaths_total = sum(:deaths)
            :py_total = sum(:py)
        end
        @rtransform :CDR = :deaths_total / :py_total
    end
    return df
end

println(calculate_CDR(kenya_df))
println(calculate_CDR(sweden_df))
println(calculate_CDR(world_df))

2×4 DataFrame
 Row │ period     deaths_total  py_total       CDR       
     │ String15   Float64       Float64        Float64   
─────┼───────────────────────────────────────────────────
   1 │ 1950-1955       769.311  32104.7        0.0239625
   2 │ 2005-2010      1962.44       1.88894e5  0.0103891
2×4 DataFrame
 Row │ period     deaths_total  py_total  CDR        
     │ String15   Float64       Float64   Float64    
─────┼───────────────────────────────────────────────
   1 │ 1950-1955       350.455   35597.8  0.00984484
   2 │ 2005-2010       456.793   45823.9  0.00996846
2×4 DataFrame
 Row │ period     deaths_total  py_total   CDR        
     │ String15   Float64       Float64    Float64    
─────┼────────────────────────────────────────────────
   1 │ 1950-1955     2.53019e5  1.3097e7   0.0193189
   2 │ 2005-2010     2.72492e5  3.33688e7  0.00816608


In [95]:
function calc_ASDR(df)::DataFrame
    df = @chain df begin
        rename("py.men" => :py_men, "py.women" => :py_women)
        @rsubset(:period == "2005-2010")
        @rtransform begin
            :ASDR = :deaths / (:py_men + :py_women)
        end 
    end
    return df
end
println(calc_ASDR(kenya_df))
println(calc_ASDR(sweden_df))
println(calc_ASDR(world_df))

15×9 DataFrame
 Row │ country  period     age      births    deaths   py_men     py_women   l_x       ASDR       
     │ String3  String15   String7  Float64   Float64  Float64    Float64    Float64   Float64    
─────┼────────────────────────────────────────────────────────────────────────────────────────────
   1 │ KEN      2005-2010  0-4         0.0    661.251  15932.6    15674.8    100000.0  0.0209208
   2 │ KEN      2005-2010  5-9         0.0     76.725  13254.0    13100.1     90959.1  0.0029113
   3 │ KEN      2005-2010  10-14       0.0     66.137  11380.9    11277.4     89360.4  0.00291889
   4 │ KEN      2005-2010  15-19    1063.63    62.441  10641.0    10575.9     88101.9  0.00294299
   5 │ KEN      2005-2010  20-24    2285.72    75.375   9707.67    9692.04    86830.0  0.00388537
   6 │ KEN      2005-2010  25-29    1868.31   105.366   8046.14    8020.33    85158.5  0.00655813
   7 │ KEN      2005-2010  30-34    1119.29   132.677   6324.06    6188.02    82430.7  0.0106039
   8 

In [101]:
kenya_asdr_df = @chain kenya_df begin
    rename("py.men" => :py_men, "py.women" => :py_women)
    @rsubset(:period == "2005-2010")
    @rtransform begin
        :ASDR = :deaths / (:py_men + :py_women)
    end
    @select(:age, :ASDR)
end

sweden_pop_ratio_df = @chain sweden_df begin
    rename("py.men" => :py_men, "py.women" => :py_women)
    @rsubset(:period == "2005-2010")
    @transform begin
        @astable begin
            :py_total = :py_men .+ :py_women
            :pop_ratio = :py_total ./ sum(:py_total)
        end
    end
    @select(:age, :pop_ratio)
end

@chain kenya_asdr_df begin
    innerjoin(sweden_pop_ratio_df, on = :age)
    @rtransform :adj_ASDR = :ASDR * :pop_ratio
    @combine(:est_CDR = sum(:adj_ASDR))
end

,est_CDR
,Float64
1,0.0232165
